# Day 5: If You Give A Seed A Fertilizer

[https://adventofcode.com/2023/day/5](https://adventofcode.com/2023/day/5)

## Description

### Part One

You take the boat and find the gardener right where you were told he would be: managing a giant "garden" that looks more to you like a farm.

"A water source? Island Island _is_ the water source!" You point out that Snow Island isn't receiving any water.

"Oh, we had to stop the water because we _ran out of sand_ to [filter](https://en.wikipedia.org/wiki/Sand_filter) it with! Can't make snow with dirty water. Don't worry, I'm sure we'll get more sand soon; we only turned off the water a few days... weeks... oh no." His face sinks into a look of horrified realization.

"I've been so busy making sure everyone here has food that I completely forgot to check why we stopped getting more sand! There's a ferry leaving soon that is headed over in that direction - it's much faster than your boat. Could you please go check it out?"

You barely have time to agree to this request when he brings up another. "While you wait for the ferry, maybe you can help us with our _food production problem_. The latest Island Island [Almanac](https://en.wikipedia.org/wiki/Almanac) just arrived and we're having trouble making sense of it."

The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil `123` and fertilizer `123` aren't necessarily related to each other.

For example:

    seeds: 79 14 55 13
    
    seed-to-soil map:
    50 98 2
    52 50 48
    
    soil-to-fertilizer map:
    0 15 37
    37 52 2
    39 0 15
    
    fertilizer-to-water map:
    49 53 8
    0 11 42
    42 0 7
    57 7 4
    
    water-to-light map:
    88 18 7
    18 25 70
    
    light-to-temperature map:
    45 77 23
    81 45 19
    68 64 13
    
    temperature-to-humidity map:
    0 69 1
    1 0 69
    
    humidity-to-location map:
    60 56 37
    56 93 4
    

The almanac starts by listing which seeds need to be planted: seeds `79`, `14`, `55`, and `13`.

The rest of the almanac contains a list of _maps_ which describe how to convert numbers from a _source category_ into numbers in a _destination category_. That is, the section that starts with `seed-to-soil map:` describes how to convert a _seed number_ (the source) to a _soil number_ (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire _ranges_ of numbers that can be converted. Each line within a map contains <span title="Don't blame me for the weird order. Blame LXC container.conf UID mappings.">three numbers</span>: the _destination range start_, the _source range start_, and the _range length_.

Consider again the example `seed-to-soil map`:

    50 98 2
    52 50 48
    

The first line has a _destination range start_ of `50`, a _source range start_ of `98`, and a _range length_ of `2`. This line means that the source range starts at `98` and contains two values: `98` and `99`. The destination range is the same length, but it starts at `50`, so its two values are `50` and `51`. With this information, you know that seed number `98` corresponds to soil number `50` and that seed number `99` corresponds to soil number `51`.

The second line means that the source range starts at `50` and contains `48` values: `50`, `51`, ..., `96`, `97`. This corresponds to a destination range starting at `52` and also containing `48` values: `52`, `53`, ..., `98`, `99`. So, seed number `53` corresponds to soil number `55`.

Any source numbers that _aren't mapped_ correspond to the _same_ destination number. So, seed number `10` corresponds to soil number `10`.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:

    seed  soil
    0     0
    1     1
    ...   ...
    48    48
    49    49
    50    52
    51    53
    ...   ...
    96    98
    97    99
    98    50
    99    51
    

With this map, you can look up the soil number required for each initial seed number:

*   Seed number `79` corresponds to soil number `81`.
*   Seed number `14` corresponds to soil number `14`.
*   Seed number `55` corresponds to soil number `57`.
*   Seed number `13` corresponds to soil number `13`.

The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find _the lowest location number that corresponds to any of the initial seeds_. To do this, you'll need to convert each seed number through other categories until you can find its corresponding _location number_. In this example, the corresponding types are:

*   Seed `79`, soil `81`, fertilizer `81`, water `81`, light `74`, temperature `78`, humidity `78`, _location `82`_.
*   Seed `14`, soil `14`, fertilizer `53`, water `49`, light `42`, temperature `42`, humidity `43`, _location `43`_.
*   Seed `55`, soil `57`, fertilizer `57`, water `53`, light `46`, temperature `82`, humidity `82`, _location `86`_.
*   Seed `13`, soil `13`, fertilizer `52`, water `41`, light `34`, temperature `34`, humidity `35`, _location `35`_.

So, the lowest location number in this example is _`35`_.

_What is the lowest location number that corresponds to any of the initial seed numbers?_


#### Solution

In [133]:
# Splitting the text into lines
almanac = open("data.txt").read().strip().split("\n")

# Dictionary to hold the results
almanac_dict = {}
current_key = None

for line in almanac:
    if ":" in line:
        # This is a new key
        current_key = line.split(":")[0].strip()
        almanac_dict[current_key] = []
    elif line.strip():
        # This is a continuation of values for the current key
        values = list(map(int, line.strip().split()))
        # Check if previous line for this key was empty to create a new list
        if not almanac_dict[current_key] or type(almanac_dict[current_key][-1]) == list:
            almanac_dict[current_key].append(values)
        else:
            almanac_dict[current_key] = [almanac_dict[current_key], values]

# The key 'seeds' should contain a single list of integers
seeds = list(map(int, almanac[0].split()[1:]))

# Removing the word 'map' from the keys in the dictionary
almanac_dict = {key.replace(" map", ""): value for key, value in almanac_dict.items()}

# Delete the seeds from the almanac dict
del almanac_dict["seeds"]

In [134]:
def x_to_y(number: int, mapping: list) -> int:
    for dest_start, source_start, _range in mapping:
        if source_start <= number < source_start + _range:
            return dest_start + (number - source_start)
    return number

In [135]:
min_location = float("inf")  # some large number
for seed in seeds:
    number = seed
    for key in almanac_dict.keys():
        number = x_to_y(number, almanac_dict[key])
    min_location = min(min_location, number)
print("Answer #1:", min_location)

Answer #1: 174137457


### Part Two

Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the `seeds:` line actually describes _ranges of seed numbers_.

The values on the initial `seeds:` line come in pairs. Within each pair, the first value is the _start_ of the range and the second value is the _length_ of the range. So, in the first line of the example above:

    seeds: 79 14 55 13

This line describes two ranges of seed numbers to be planted in the garden. The first range starts with seed number `79` and contains `14` values: `79`, `80`, ..., `91`, `92`. The second range starts with seed number `55` and contains `13` values: `55`, `56`, ..., `66`, `67`.

Now, rather than considering four seed numbers, you need to consider a total of _27_ seed numbers.

In the above example, the lowest location number can be obtained from seed number `82`, which corresponds to soil `84`, fertilizer `84`, water `84`, light `77`, temperature `45`, humidity `46`, and _location `46`_. So, the lowest location number is _`46`_.

Consider all of the initial seed numbers listed in the ranges on the first line of the almanac. _What is the lowest location number that corresponds to any of the initial seed numbers?_


#### Solution (Brute Force)

In [136]:
min_location = 1e6  # some large number
for idx in range(0, len(seeds), 2):
    seed_start = seeds[idx]
    seed_end = seeds[idx] + seeds[int(idx + 1)]

    for seed in range(seed_start, seed_end):
        number = seed
        for key in almanac_dict.keys():
            number = x_to_y(number, almanac_dict[key])
        min_location = min(min_location, number)

print("Answer #2", min_location)

KeyboardInterrupt: 

In [ ]:
min_locations = []

for idx in range(0, len(seeds), 2):
    seed_start = seeds[idx]
    seed_end = seed_start + seeds[idx + 1]

    # Initially, the minimum number in the range
    min_number = seed_start

    # Apply each mapping to find the minimum number at each stage
    for mapping in almanac_dict.values():
        temp_min = float("inf")
        for dest_start, source_start, _range in mapping:
            source_end = source_start + _range

            if source_start <= min_number < source_end:
                # If the current minimum number is within this mapping rule
                temp_min = min(temp_min, dest_start + (min_number - source_start))
            elif min_number < source_start:
                # If the current minimum number is smaller than this mapping rule
                temp_min = min(temp_min, dest_start)

        # Update min_number for the next stage
        min_number = temp_min if temp_min != float("inf") else min_number

    min_locations.append(min_number)


# Example usage
# seeds = [list of seed numbers and ranges]
# almanac_dict = {stage_name: mapping_rules, ...}
# optimized_min_location = find_min_location_optimized(seeds, almanac_dict)
# optimized_min_location